# Load the CIFAR-10 Data

In [1]:
import numpy as np
import tensorflow as tf
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        cifar_dict = pickle.load(fo, encoding='bytes')
    return cifar_dict

train_batch_1 = unpickle('./cifar-10-batches-py/data_batch_1')
train_batch_2 = unpickle('./cifar-10-batches-py/data_batch_2')
train_batch_3 = unpickle('./cifar-10-batches-py/data_batch_3')
train_batch_4 = unpickle('./cifar-10-batches-py/data_batch_4')
train_batch_5 = unpickle('./cifar-10-batches-py/data_batch_5')
test_batch = unpickle('./cifar-10-batches-py/test_batch')

train_dicts = [train_batch_1, train_batch_2, train_batch_3, train_batch_4, train_batch_5]

In [2]:
train_data = np.concatenate([np.array(tdict[b'data']) for tdict in train_dicts], axis=0).reshape((-1, 3, 32, 32)).transpose([0, 2, 3, 1])
train_label = np.concatenate([np.array(tdict[b'labels']) for tdict in train_dicts], axis=0)

test_data = np.array(test_batch[b'data']).reshape((-1, 3, 32, 32)).transpose([0, 2, 3, 1])
test_label = np.array(test_batch[b'labels'])

print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


# Load Pre-trained ResNet50 and Perform Preprocessing

In [3]:
resnet_50_model = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')

upstream_input = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.layers.UpSampling2D(size=(7,7))(upstream_input)
x = resnet_50_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)

upstream_model = tf.keras.Model(inputs=upstream_input, outputs=x)
upstream_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-tra

In [4]:
downstream_model = tf.keras.Sequential()
downstream_model.add(tf.keras.Input(shape=(2048,)))
# downstream_model.add(tf.keras.layers.Dense(1024, activation="relu"))
downstream_model.add(tf.keras.layers.Dense(512, activation="relu"))
# downstream_model.add(tf.keras.layers.Dropout(0.1))
downstream_model.add(tf.keras.layers.Dense(10))
downstream_model.build()
# downstream_model.compile(optimizer='SGD', metrics=['categorical_accuracy'], loss='categorical_crossentropy')
downstream_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 1,054,218
Trainable params: 1,054,218
Non-trainable params: 0
_________________________________________________________________


In [5]:
whole_model_input = tf.keras.Input(shape=(32, 32, 3))
whole_output = upstream_model(whole_model_input)
whole_output = downstream_model(whole_output)
whole_model = tf.keras.Model(inputs=whole_model_input, outputs=whole_output)

train_data_ready = tf.keras.applications.resnet50.preprocess_input(train_data)
test_data_ready = tf.keras.applications.resnet50.preprocess_input(test_data)

train_y_onehot = tf.one_hot(train_label, 10)
test_y_onehot = tf.one_hot(test_label, 10)

whole_model.summary()
whole_model.compile(optimizer='SGD', metrics=['categorical_accuracy'], loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 model (Functional)          (None, 2048)              23587712  
                                                                 
 sequential (Sequential)     (None, 10)                1054218   
                                                                 
Total params: 24,641,930
Trainable params: 24,588,810
Non-trainable params: 53,120
_________________________________________________________________


In [6]:
whole_model.fit(x=train_data_ready,
                y=train_y_onehot,
                batch_size=64,
                epochs=3,
                validation_split=0.2)

Epoch 1/3
625/625 [==============================] - 111s 170ms/step - loss: 0.4447 - categorical_accuracy: 0.8525 - val_loss: 0.2157 - val_categorical_accuracy: 0.9261
Epoch 2/3
625/625 [==============================] - 106s 170ms/step - loss: 0.1126 - categorical_accuracy: 0.9636 - val_loss: 0.1884 - val_categorical_accuracy: 0.9376
Epoch 3/3
625/625 [==============================] - 107s 170ms/step - loss: 0.0365 - categorical_accuracy: 0.9899 - val_loss: 0.1798 - val_categorical_accuracy: 0.9470


In [7]:
whole_model.evaluate(x=test_data_ready, y=test_y_onehot)

313/313 [==============================] - 9s 28ms/step - loss: 0.1726 - categorical_accuracy: 0.9460


[0.17258386313915253, 0.9459999799728394]

In [8]:
test_data_embedded = upstream_model.predict(test_data_ready)
test_reg_labels = downstream_model.predict(test_data_embedded)


In [9]:

print(test_data_embedded, test_reg_labels)
print(test_data_embedded.shape, test_reg_labels.shape)

[[0.7106468  1.0221604  0.8439132  ... 0.02926966 0.36094007 1.5012412 ]
 [0.2524052  0.         0.16202419 ... 0.25519451 0.         1.3707749 ]
 [0.0033594  0.13601582 0.45645764 ... 0.05592443 0.0251136  0.38941503]
 ...
 [0.48810056 0.65276015 0.4458373  ... 0.         0.00387705 0.74837536]
 [0.7428815  0.5042405  0.         ... 1.4975706  0.30778193 0.61596483]
 [0.05659481 0.761282   0.05134956 ... 3.2003448  0.910273   0.13906668]] [[-3.6417968  -1.0284551   0.702696   ... -4.835945   -1.1543816
  -3.2433705 ]
 [ 1.3965961   3.303125   -3.4402146  ... -2.8886669  14.258844
  -3.6290505 ]
 [ 1.7380804   4.4006333  -3.4718938  ... -2.7244823   9.778114
  -0.95517975]
 ...
 [-2.529127   -2.9129558  -0.19578713 ...  0.42118183 -1.6554612
  -1.9873008 ]
 [ 3.061164   10.510604   -0.16372842 ... -1.7127796  -1.5226618
  -0.8852281 ]
 [-2.8599532  -2.1474118  -0.45818564 ... 10.996795   -3.697093
  -2.1767864 ]]
(10000, 2048) (10000, 10)


In [10]:
np.savez('./data/resnet50_data.npz', features=test_data_embedded, labels=test_reg_labels)